# Install

In [1]:
pip install pybaseball

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 45.8 MB/s eta 0:00:00


# Imports

In [29]:
import os
import pandas as pd
import numpy as np
import pybaseball as pyb
from pybaseball import batting_stats

# Pull Data and Save Data

In [30]:
START = 2002
END = 2022

In [31]:
batting = batting_stats(START, END, qual=200)

In [32]:
batting.to_csv('batting.csv')

In [33]:
# we want players with at least two seasons -- remove groups with one season of data
# IDfg is unique player ID
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1)

In [34]:
batting

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN,12.7
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN,11.9
8,15640,2022,Aaron Judge,NYY,30,157,570,696,177,87,...,118.4,246.0,0.609,404,0.169,0.287,NaN,NaN,NaN,11.4
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN,10.2
15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,110.6,217.0,0.500,434,0.220,0.270,NaN,NaN,NaN,10.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7042,9272,2018,Chris Davis,BAL,32,128,470,522,79,51,...,111.8,113.0,0.401,282,0.174,0.316,NaN,NaN,NaN,-2.6
6535,45,2012,Rod Barajas,PIT,36,104,321,361,66,44,...,NaN,0.0,NaN,0,0.147,0.258,NaN,NaN,NaN,-2.6
6673,319,2011,Adam Dunn,CHW,31,122,415,496,66,39,...,NaN,0.0,NaN,0,0.169,0.295,NaN,NaN,NaN,-2.9
6988,620,2002,Neifi Perez,KCR,29,145,554,585,131,104,...,NaN,NaN,NaN,0,0.130,0.187,NaN,NaN,NaN,-2.9


# Prep the target variable

In [42]:
# set up target variable - WAR for next season
def next_season(player):
  """take data for single player and backfill WAR value for next season"""
  player = player.sort_values("Season")
  player["Next_WAR"] = player["WAR"].shift(-1)
  return player

# split dataframe by playerId and compute next season WAR for each player
batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

<ipython-input-42-c59be7a85902>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting = batting.groupby("IDfg", group_keys=False).apply(next_season)


In [43]:
batting[["Name","Season","WAR","Next_WAR"]]

,Name,Season,WAR,Next_WAR
5562,Alfredo Amezaga,2006,1.1,2.0
5006,Alfredo Amezaga,2007,2.0,1.2
5252,Alfredo Amezaga,2008,1.2,NaN
1169,Garret Anderson,2002,3.7,5.1
864,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
6002,Owen Miller,2022,0.7,NaN
4881,Andrew Vaughn,2021,-0.2,-0.5
3377,Andrew Vaughn,2022,-0.5,NaN
6620,Ha-seong Kim,2021,0.4,3.6


# Clean the data

In [44]:
# we can also impute these values
# we are counting missing values in each column
null_count = batting.isnull().sum()
null_count

,0
IDfg,0
Season,0
Name,0
Team,0
Age,0
...,...
xBA,6754
xSLG,6754
xwOBA,6754
L-WAR,0


In [45]:
complete_cols = list(batting.columns[null_count==0])
complete_cols

['IDfg',
 'Season',
 'Name',
 'Team',
 'Age',
 'G',
 'AB',
 'PA',
 'H',
 '1B',
 '2B',
 '3B',
 'HR',
 'R',
 'RBI',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'SF',
 'SH',
 'GDP',
 'SB',
 'CS',
 'AVG',
 'GB',
 'FB',
 'LD',
 'IFFB',
 'Pitches',
 'Balls',
 'Strikes',
 'IFH',
 'BU',
 'BUH',
 'BB%',
 'K%',
 'BB/K',
 'OBP',
 'SLG',
 'OPS',
 'ISO',
 'BABIP',
 'GB/FB',
 'LD%',
 'GB%',
 'FB%',
 'IFFB%',
 'HR/FB',
 'IFH%',
 'BUH%',
 'wOBA',
 'wRAA',
 'wRC',
 'Bat',
 'Rep',
 'Pos',
 'RAR',
 'WAR',
 'Dol',
 'Spd',
 'wRC+',
 'WPA',
 '-WPA',
 '+WPA',
 'RE24',
 'REW',
 'pLI',
 'PH',
 'WPA/LI',
 'Clutch',
 'FB% (Pitch)',
 'FBv',
 'SL%',
 'SLv',
 'CB%',
 'CBv',
 'CH%',
 'CHv',
 'wFB',
 'wSL',
 'wCB',
 'wCH',
 'wFB/C',
 'wSL/C',
 'wCB/C',
 'wCH/C',
 'O-Swing%',
 'Z-Swing%',
 'Swing%',
 'O-Contact%',
 'Z-Contact%',
 'Contact%',
 'Zone%',
 'F-Strike%',
 'SwStr%',
 'BsR',
 'Def',
 'wSB',
 'UBR',
 'Age Rng',
 'Off',
 'Lg',
 'wGDP',
 'Pull%',
 'Cent%',
 'Oppo%',
 'Soft%',
 'Med%',
 'Hard%',
 'TTO%',
 'AVG+',
 'BB%+',
 'K

In [46]:
# avoid copy warnings later
batting = batting[complete_cols + ["Next_WAR"]].copy()

In [47]:
batting.dtypes

,0
IDfg,int64
Season,int64
Name,object
Team,object
Age,int64
...,...
Events,int64
CStr%,float64
CSW%,float64
L-WAR,float64


In [48]:
# handle string values
batting.dtypes[batting.dtypes == "object"]

,0
Name,object
Team,object
Dol,object
Age Rng,object


In [49]:
# for now delete these columns -- could use age in the future
del batting['Dol']
del batting['Age Rng']
batting['team_code'] = batting['Team'].astype("category").cat.codes # ordinal encoding ... careful here

In [50]:
batting_full = batting.copy()
batting = batting.dropna().copy() # drop where next war is null

# Model

In [51]:
# run feature selector to help model optimize accuracy
# define model (ridge regression model)

from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit # used in feature selector

In [52]:
# play around with lambda / alpha parameter, the higher it is the more it penalizes Ridge Coeff and avoids overfitting
rr = Ridge(alpha=1)
split = TimeSeriesSplit(n_splits=3) # make predictions for three parts because we have time series data
# go through all the features, find the best one, so on -- direction forward, remove features for direction = backward
sfs = SequentialFeatureSelector(rr, n_features_to_select=20, direction="forward", cv=split, n_jobs=4)

In [53]:
removed_cols = ['Next_WAR', 'Name', 'Team','IDfg', 'Season']
selected_cols = batting.columns[~batting.columns.isin(removed_cols)]

In [54]:
# with ridge regression we have to scale data so mean is 1 and regreesion is 1
from sklearn.preprocessing import MinMaxScaler # puts all vallus betweeen 0-1
scaler = MinMaxScaler()
batting.loc[:, selected_cols] = scaler.fit_transform(batting[selected_cols])

<ipython-input-54-d8e7bfc33daf>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.34615385 0.38461538 0.42307692 ... 0.19230769 0.15384615 0.23076923]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batting.loc[:, selected_cols] = scaler.fit_transform(batting[selected_cols])
<ipython-input-54-d8e7bfc33daf>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.73504274 0.74358974 0.95726496 ... 0.11965812 0.69230769 0.60683761]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  batting.loc[:, selected_cols] = scaler.fit_transform(batting[selected_cols])
<ipython-input-54-d8e7bfc33daf>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.31294964 0.43165468 0.85971223 ... 0.0557554  0.46223022 0.1

In [56]:
batting.describe()
# columns now have been scaled between 0 and 1

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,L-WAR,Next_WAR,team_code
count,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,...,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000,5575.000000
mean,5366.785830,2011.163229,0.360600,0.652755,0.478666,0.480943,0.365973,0.290481,0.399279,0.103459,...,0.403164,0.410923,0.511026,0.478646,0.172991,0.498932,0.545898,0.322039,1.793112,0.474128
std,5133.255295,5.612014,0.147476,0.255929,0.242481,0.262290,0.182585,0.138786,0.171732,0.105891,...,0.131213,0.121082,0.130359,0.133992,0.273858,0.137180,0.120701,0.122147,1.981100,0.305105
min,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.100000,0.000000
25%,1131.500000,2006.000000,0.269231,0.478632,0.275180,0.257785,0.211207,0.179245,0.258621,0.043478,...,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.234177,0.400000,0.205882
50%,3531.000000,2011.000000,0.346154,0.709402,0.505396,0.508651,0.370690,0.283019,0.379310,0.086957,...,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,0.303797,1.500000,0.470588
75%,9015.000000,2016.000000,0.461538,0.871795,0.688849,0.710208,0.508621,0.391509,0.517241,0.130435,...,0.488722,0.483146,0.594203,0.564626,0.346411,0.591489,0.625551,0.392405,2.900000,0.735294
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [57]:
# fit selector to data -- fits best 20 features to give us best results for ridge regression
# then extract list of predictors from feature selector
sfs.fit(batting[selected_cols], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [60]:
predictors = list(selected_cols[sfs.get_support()])
# trues are columns we want to select

In [62]:
def backtest(data, model,predictors, start=5, step=1):
  """what we use to validate data in cross validation -- in timeseries
  we don't want to use data from a future season to predict past season.
  we only want past data to predict future data for timeseries cross validation
  ie. use 2002 to predict 2003, then use 2002, 2003 to predict 2004, etc.
  """

  all_predictions = []
  # get sorted years / seasons
  years = sorted(data["Season"].unique())

  for i in range(start, len(years), step):
    current_year = years[i]
    # split data
    train = data[data["Season"] < current_year]
    test = data[data["Season"] == current_year]
    # define label and data
    x_train = train[predictors]
    x_test = test[predictors]
    y_train = train["Next_WAR"]
    y_test = test["Next_WAR"]
    # fit and predict
    model.fit(x_train, y_train)
    preds = model.predict(x_test)
    # convert to series so easier to work with (column of data)
    preds = pd.Series(preds, index=y_test.index)
    # combine prediction with actuals
    combined = pd.concat([y_test, preds], axis=1)
    # assign column names
    combined.columns = ["actual", "prediction"]
    # append to all predictions - list of dataframes each is a predictions for a different season
    all_predictions.append(combined)
    # concat on axis=0 (vertically one long dataframe)
  return pd.concat(all_predictions)

In [64]:
predictions = backtest(batting, rr, predictors)
predictions

,actual,prediction
5006,1.2,1.405835
1925,1.4,0.716105
3102,-0.1,0.457908
5797,0.6,0.979155
1109,4.8,2.214873
...,...,...
1914,2.2,2.752929
5875,0.8,2.083841
7032,0.7,1.583827
4881,-0.5,1.819488


# Evaluate

In [65]:
# use summary stats to evaluate performance
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions["actual"], predictions["prediction"])

2.736105953760221

In [66]:
batting["Next_WAR"].describe()

,Next_WAR
count,5575.000000
mean,1.793112
std,1.981100
min,-3.100000
25%,0.400000
50%,1.500000
75%,2.900000
max,11.900000


We probably want MSE to be lower than the standard dev ideally (typical rule of thumb) -- indicates our model is doing better than randomly guessing.

In [67]:
2.736105953760221 ** 0.5
# take away square root and we see the value is slightly lower which tells us our model is ok.

1.654117877831027

# Improving Accuracy

Right now we are only telling the model what happened in the current season, we might want to tell the model what happened in past seasons -- might be useful to see if the player is in decline

In [74]:
# example:
ga = batting[batting["IDfg"] == 2]
ga["player_season"] = range(0,ga.shape[0])
# defines groups and finds correlation within each group that expanding creates
# two index value to select one number
ga[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"),"WAR"]

<ipython-input-74-fbb4ce47011b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ga["player_season"] = range(0,ga.shape[0])


,,WAR
1169,player_season,NaN
864,player_season,1.000000
2569,player_season,-0.661143
4187,player_season,-0.836562
3964,player_season,-0.836312
1925,player_season,-0.692192
3346,player_season,-0.595013


In [75]:
def group_averages(df):
  """did our player play better than an average player"""
  return df["WAR"] / df["WAR"].mean()

In [78]:
def player_history(df):
  df = df.sort_values("Season")
  # indicates which season it is for the player
  df["player_season"] = range(0,df.shape[0])
  df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
  # 1 implies 1:1 correlation
  df["war_corr"].fillna(1, inplace=True)
  # brings prevous season up to current
  df["war_diff"] = df["WAR"] / df["WAR"].shift(1)
  df["war_diff"].fillna(1,inplace=True)
  df["war_diff"][df["war_diff"] == np.inf] = 1
  return df

batting = batting.groupby("IDfg", group_keys = False).apply(player_history)


Streaming output truncated to the last 5000 lines.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["war_corr"].fillna(1, inplace=True)
<ipython-input-78-613bafb602f7>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["war_diff"].fillna(1,inplace=True)
<ipython-input-78-613bafb602f7>:11: FutureWarning: ChainedAssignmentError: behaviour will change in pan

In [76]:
# how average player performed
batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)

<ipython-input-76-c2659c141f07>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)


In [79]:
new_predictors = predictors + ["player_season","war_corr", "war_season", "war_diff"]

In [82]:
predictons = backtest(batting, rr, new_predictors)
mean_squared_error(predictions["actual"], predictions["prediction"])

2.736105953760221

# Evaluate Predictors

In [88]:
pd.Series(rr.coef_, index=new_predictors).sort_values()
# use this to diagnose which predictors are best

,0
Age,-2.712560
BABIP,-1.937445
WAR,-1.831011
SLG+,-1.381371
Soft%+,-1.322751
BU,-1.125847
SO,-0.896876
PH,-0.746790
WPA,-0.552493
CH%,-0.289703


In [90]:
diff = predictions["actual"] - predictions["prediction"]

In [92]:
merged = predictions.merge(batting,left_index=True, right_index=True)
merged["diff"] = predictions["actual"] - predictions["prediction"].abs()
# merged[['IDfg', "Season",'Name', 'WAR', 'Next_WAR']]]

,actual,prediction,IDfg,Season,Name,Team,Age,G,AB,PA,...,CStr%,CSW%,L-WAR,Next_WAR,team_code,war_season,player_season,war_corr,war_diff,diff
5006,1.2,1.405835,1,2007,Alfredo Amezaga,FLA,0.384615,0.743590,0.431655,0.429066,...,0.527660,0.396476,0.322785,1.2,0.352941,0.998259,1,1.000000,1.214286,-0.205835
1925,1.4,0.716105,2,2007,Garret Anderson,LAA,0.615385,0.529915,0.462230,0.432526,...,0.442553,0.480176,0.284810,1.4,0.441176,0.880816,5,-0.692192,1.406250,0.683895
3102,-0.1,0.457908,10,2007,David Eckstein,STL,0.500000,0.606838,0.492806,0.491349,...,0.676596,0.436123,0.240506,-0.1,0.852941,0.743801,5,-0.694330,0.826087,-0.557908
5797,0.6,0.979155,11,2007,Darin Erstad,CHW,0.538462,0.350427,0.269784,0.254325,...,0.765957,0.691630,0.240506,0.6,0.205882,0.743801,4,-0.828562,0.791667,-0.379155
1109,4.8,2.214873,15,2007,Troy Glaus,TOR,0.423077,0.589744,0.404676,0.442907,...,0.634043,0.704846,0.367089,4.8,0.970588,1.135274,5,0.231396,0.892308,2.585127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,2.2,2.752929,23667,2021,Wander Franco,TBR,0.038462,0.205128,0.217626,0.186851,...,0.391489,0.352423,0.348101,2.2,0.911765,1.062206,0,1.000000,1.000000,-0.552929
5875,0.8,2.083841,24618,2021,Ryan Jeffers,MIN,0.192308,0.333333,0.192446,0.160900,...,0.514894,0.788546,0.240506,0.8,0.558824,0.830452,0,1.000000,1.000000,-1.283841
7032,0.7,1.583827,24655,2021,Owen Miller,CLE,0.192308,0.119658,0.055755,0.003460,...,0.548936,0.700441,0.139241,0.7,0.264706,0.463508,0,1.000000,1.000000,-0.883827
4881,-0.5,1.819488,26197,2021,Andrew Vaughn,CHW,0.153846,0.692308,0.462230,0.465398,...,0.570213,0.651982,0.170886,-0.5,0.205882,0.560072,0,1.000000,1.000000,-2.319488


# Next steps

- Use better data (ie. minor league data, etc.)
- Use a different model
- Use different selection strategies
- Feature engineering